In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/physionet-ecg-image-digitization/sample_submission.parquet
/kaggle/input/physionet-ecg-image-digitization/train.csv
/kaggle/input/physionet-ecg-image-digitization/test.csv
/kaggle/input/physionet-ecg-image-digitization/test/2352854581.png
/kaggle/input/physionet-ecg-image-digitization/test/1053922973.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0005.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0006.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0011.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0004.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893.csv
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0012.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0003.png
/kaggle/input/physionet-ecg-image-digitization/train/735384893/735384893-0001.png
/kaggle/input/physionet-ecg-i

In [2]:
# ECG Image to Time-Series Extraction - FULL FIXED VERSION
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import warnings
import gc
import multiprocessing
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

# Device setup
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    torch.set_num_threads(multiprocessing.cpu_count())
    print(f"CPU: {multiprocessing.cpu_count()} cores")

print("="*80)
print("ECG IMAGE TO TIME-SERIES - FULL FIXED VERSION")
print("="*80)

# ============================================================================
# DATA EXPLORATION
# ============================================================================

def explore_data():
    base_path = '/kaggle/input/physionet-ecg-image-digitization'
    
    print("\n" + "="*80)
    print("DATA VERIFICATION")
    print("="*80)
    
    # Load CSVs
    train_csv_path = os.path.join(base_path, 'train.csv')
    test_csv_path = os.path.join(base_path, 'test.csv')
    
    train_df = None
    test_df = None
    
    if os.path.exists(train_csv_path):
        train_df = pd.read_csv(train_csv_path)
        print(f"\ntrain.csv loaded: {train_df.shape}")
        print(f"Columns: {list(train_df.columns)}")
    
    if os.path.exists(test_csv_path):
        test_df = pd.read_csv(test_csv_path)
        print(f"\ntest.csv loaded: {test_df.shape}")
        print(f"Columns: {list(test_df.columns)}")
        print(f"Sample:\n{test_df.head()}")
    
    # Check train folder
    train_path = os.path.join(base_path, 'train')
    patient_dirs = []
    if os.path.exists(train_path):
        patient_dirs = sorted([d for d in os.listdir(train_path) 
                              if os.path.isdir(os.path.join(train_path, d))])
        print(f"\nTotal patient folders: {len(patient_dirs)}")
    
    # Check test folder
    test_path = os.path.join(base_path, 'test')
    test_images = []
    if os.path.exists(test_path):
        test_images = sorted([f for f in os.listdir(test_path) if f.endswith('.png')])
        print(f"\n🔍 TEST IMAGES FOUND: {len(test_images)}")
        for img in test_images:
            print(f"  - {img}")
    
    return base_path, train_df, test_df, patient_dirs, test_images

base_path, train_df, test_df, patient_dirs, test_images = explore_data()

# ============================================================================
# DATASET
# ============================================================================

class ECGDataset(Dataset):
    def __init__(self, base_path, patient_ids, transform=None, is_test=False, max_len=5000):
        self.base_path = base_path
        self.transform = transform
        self.is_test = is_test
        self.max_len = max_len
        self.samples = []
        
        if is_test:
            test_dir = os.path.join(base_path, 'test')
            
            if isinstance(patient_ids, list):
                for img_file in patient_ids:
                    if img_file.endswith('.png'):
                        img_path = os.path.join(test_dir, img_file)
                        patient_id = img_file.replace('.png', '')
                    else:
                        img_path = os.path.join(test_dir, f"{img_file}.png")
                        patient_id = img_file
                    
                    if os.path.exists(img_path):
                        self.samples.append({
                            'patient_id': patient_id,
                            'image_path': img_path
                        })
        else:
            train_dir = os.path.join(base_path, 'train')
            
            if isinstance(patient_ids, pd.DataFrame):
                patient_ids = patient_ids['patient_id'].unique() if 'patient_id' in patient_ids.columns else patient_ids.iloc[:, 0].unique()
            
            for pid in patient_ids:
                p_folder = os.path.join(train_dir, str(pid))
                if not os.path.exists(p_folder):
                    continue
                
                images = sorted([f for f in os.listdir(p_folder) if f.endswith('.png')])
                csv_file = os.path.join(p_folder, f"{pid}.csv")
                
                if os.path.exists(csv_file):
                    for img in images:
                        self.samples.append({
                            'patient_id': pid,
                            'image_path': os.path.join(p_folder, img),
                            'csv_path': csv_file
                        })
        
        print(f"  📊 Total samples loaded: {len(self.samples)}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Load image
        image = cv2.imread(sample['image_path'])
        if image is None:
            print(f"⚠️  Failed to load: {sample['image_path']}")
            image = np.zeros((512, 512, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, sample['patient_id']
        
        # Load target from CSV
        target = torch.zeros(12, self.max_len)
        
        if 'csv_path' in sample and os.path.exists(sample['csv_path']):
            try:
                df = pd.read_csv(sample['csv_path'])
                df = df.fillna(0.0)
                
                for lead_idx in range(min(12, len(df.columns))):
                    lead_data = df.iloc[:, lead_idx].values.astype(np.float32)
                    length = min(len(lead_data), self.max_len)
                    target[lead_idx, :length] = torch.from_numpy(lead_data[:length])
                    
            except Exception as e:
                print(f"Error loading {sample['csv_path']}: {e}")
        
        return image, target, sample['patient_id']


def get_transforms(is_train=True, img_size=(384, 384)):
    if is_train:
        return transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(img_size),
            transforms.RandomRotation(degrees=2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# ============================================================================
# MODEL
# ============================================================================

class ECGEncoder(nn.Module):
    def __init__(self, latent_dim=256):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.fc = nn.Linear(256, latent_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


class ECGDecoder(nn.Module):
    def __init__(self, latent_dim=256, seq_len=5000, num_leads=12):
        super().__init__()
        
        self.seq_len = seq_len
        self.fc_init = nn.Linear(latent_dim, 128)
        self.lstm1 = nn.LSTM(128, 256, batch_first=True, bidirectional=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(512, 128, batch_first=True, bidirectional=True)
        self.fc_out = nn.Linear(256, num_leads)
    
    def forward(self, x):
        x = self.fc_init(x)
        x = x.unsqueeze(1).repeat(1, self.seq_len, 1)
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.fc_out(x)
        x = x.permute(0, 2, 1)
        return x


class ECGModel(nn.Module):
    def __init__(self, latent_dim=256, seq_len=5000, num_leads=12):
        super().__init__()
        self.encoder = ECGEncoder(latent_dim)
        self.decoder = ECGDecoder(latent_dim, seq_len, num_leads)
    
    def forward(self, x):
        latent = self.encoder(x)
        output = self.decoder(latent)
        return output

# ============================================================================
# SNR CALCULATION
# ============================================================================

def calculate_snr_fixed(pred, target, eps=1e-10):
    pred = pred.detach().cpu().numpy()
    target = target.detach().cpu().numpy()
    
    snr_values = []
    
    for i in range(pred.shape[0]):
        p = pred[i] - np.mean(pred[i], axis=1, keepdims=True)
        t = target[i] - np.mean(target[i], axis=1, keepdims=True)
        
        signal_power = np.sum(t ** 2) + eps
        noise_power = np.sum((p - t) ** 2) + eps
        snr_db = 10 * np.log10(signal_power / noise_power)
        
        if not np.isnan(snr_db) and not np.isinf(snr_db):
            snr_values.append(snr_db)
    
    return np.mean(snr_values) if snr_values else float('nan')

# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================

def train_epoch(model, loader, criterion, optimizer, device, scaler=None):
    model.train()
    total_loss = 0
    total_snr = 0
    num_batches = 0
    
    pbar = tqdm(loader, desc='Training')
    
    for batch_idx, batch in enumerate(pbar):
        try:
            images, targets, _ = batch
            images = images.to(device)
            targets = targets.to(device)
            
            if torch.isnan(images).any() or torch.isnan(targets).any():
                continue
            
            optimizer.zero_grad()
            
            if scaler is not None:
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    min_len = min(outputs.shape[2], targets.shape[2])
                    outputs = outputs[:, :, :min_len]
                    targets = targets[:, :, :min_len]
                    loss = criterion(outputs, targets)
                
                if torch.isnan(loss):
                    continue
                
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                min_len = min(outputs.shape[2], targets.shape[2])
                outputs = outputs[:, :, :min_len]
                targets = targets[:, :, :min_len]
                loss = criterion(outputs, targets)
                
                if torch.isnan(loss):
                    continue
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
            
            snr = calculate_snr_fixed(outputs, targets)
            
            if not np.isnan(snr):
                total_loss += loss.item()
                total_snr += snr
                num_batches += 1
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'snr': f'{snr:.2f}' if not np.isnan(snr) else 'NaN'
            })
            
        except Exception as e:
            print(f"❌ Error in batch {batch_idx}: {e}")
            continue
    
    if num_batches == 0:
        return float('nan'), float('nan')
    
    return total_loss / num_batches, total_snr / num_batches


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_snr = 0
    num_batches = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        
        for batch in pbar:
            try:
                images, targets, _ = batch
                images = images.to(device)
                targets = targets.to(device)
                
                outputs = model(images)
                min_len = min(outputs.shape[2], targets.shape[2])
                outputs = outputs[:, :, :min_len]
                targets = targets[:, :, :min_len]
                
                loss = criterion(outputs, targets)
                
                if torch.isnan(loss):
                    continue
                
                snr = calculate_snr_fixed(outputs, targets)
                
                if not np.isnan(snr):
                    total_loss += loss.item()
                    total_snr += snr
                    num_batches += 1
                
                pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'snr': f'{snr:.2f}' if not np.isnan(snr) else 'NaN'
                })
                
            except Exception as e:
                continue
    
    if num_batches == 0:
        return float('nan'), float('nan')
    
    return total_loss / num_batches, total_snr / num_batches

# ============================================================================
# SERIES CONVERSION FUNCTIONS
# ============================================================================

def series_dict(series):
    """Convert 4-row series to 12-lead dictionary"""
    d = {}
    # Row 0: I, aVR, V1, V4
    # Row 1: II, aVL, V2, V5
    # Row 2: III, aVF, V3, V6
    # Row 3: II (long)
    
    for l in range(3):
        lead_names = [
            ['I',   'aVR', 'V1', 'V4'],
            ['II',  'aVL', 'V2', 'V5'],
            ['III', 'aVF', 'V3', 'V6'],
        ][l]
        split = np.array_split(series[l], 4)
        for (k, s) in zip(lead_names, split):
            d[k] = s
    
    d['II'] = series[3]  # Use long II lead
    
    return d

# ============================================================================
# SUBMISSION GENERATOR
# ============================================================================

def generate_submission_correct(model, base_path, device):
    """Generate submission matching the exact expected format"""
    print("\n" + "="*80)
    print("GENERATING SUBMISSION - CORRECT FORMAT")
    print("="*80)
    
    # Load test.csv to get required format
    test_csv_path = os.path.join(base_path, 'test.csv')
    test = pd.read_csv(test_csv_path)
    test['id'] = test['id'].astype(str)
    
    print(f"\nTest CSV Info:")
    print(f"  Shape: {test.shape}")
    print(f"  Columns: {list(test.columns)}")
    print(f"\nSample:")
    print(test.head(15))
    
    test_dir = os.path.join(base_path, 'test')
    test_ids = test['id'].unique().tolist()
    
    print(f"\n📊 Processing {len(test_ids)} test images")
    
    # Create test dataset
    test_dataset = ECGDataset(base_path, test_ids,
                             transform=get_transforms(False, (384, 384)),
                             is_test=True, max_len=5000)
    
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False,
                            num_workers=0, pin_memory=torch.cuda.is_available())
    
    model.eval()
    
    # Store predictions for each patient
    predictions = {}
    
    print("\n🔄 Running inference...")
    with torch.no_grad():
        for images, patient_ids in tqdm(test_loader, desc='Inference'):
            images = images.to(device)
            outputs = model(images).cpu().numpy()  # Shape: [1, 12, 5000]
            
            patient_id = patient_ids[0]
            
            # Model outputs 12 leads directly
            # Need to convert to series format (4 rows)
            series = np.zeros((4, 5000))
            
            # Reconstruct 4-row format from 12 leads
            # Row 0: I, aVR, V1, V4 (concatenated)
            # Row 1: II, aVL, V2, V5 (concatenated)
            # Row 2: III, aVF, V3, V6 (concatenated)
            # Row 3: II (full length)
            
            lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
            
            # Extract each lead from model output
            lead_data = {}
            for idx, lead_name in enumerate(lead_order):
                lead_data[lead_name] = outputs[0, idx, :]
            
            # Concatenate into 4-row format
            quarter_len = 5000 // 4
            series[0] = np.concatenate([lead_data['I'][:quarter_len], 
                                       lead_data['aVR'][:quarter_len],
                                       lead_data['V1'][:quarter_len], 
                                       lead_data['V4'][:quarter_len]])
            series[1] = np.concatenate([lead_data['II'][:quarter_len], 
                                       lead_data['aVL'][:quarter_len],
                                       lead_data['V2'][:quarter_len], 
                                       lead_data['V5'][:quarter_len]])
            series[2] = np.concatenate([lead_data['III'][:quarter_len], 
                                       lead_data['aVF'][:quarter_len],
                                       lead_data['V3'][:quarter_len], 
                                       lead_data['V6'][:quarter_len]])
            series[3] = lead_data['II']  # Long II lead
            
            predictions[patient_id] = series
            print(f"  ✓ Processed {patient_id}: shape {series.shape}")
    
    # Generate submission rows
    print("\n📝 Generating submission rows...")
    res = []
    gb = test.groupby('id')
    
    for i, (sample_id, df) in enumerate(tqdm(gb, desc='Creating submission')):
        if sample_id not in predictions:
            print(f"  ⚠️ Missing prediction for {sample_id}, using zeros")
            series = np.zeros((4, 5000))
        else:
            series = predictions[sample_id]
        
        # Convert series to dictionary of 12 leads
        d_series = series_dict(series)
        
        # Process each row in test.csv for this patient
        for _, row in df.iterrows():
            lead = row['lead']
            num_rows = int(row['number_of_rows'])
            
            # Get signal for this lead
            s = d_series.get(lead, np.zeros(num_rows))
            
            # Resample if length doesn't match
            if len(s) != num_rows:
                x_old = np.linspace(0, 1, len(s))
                x_new = np.linspace(0, 1, num_rows)
                s = np.interp(x_new, x_old, s)
            
            # Create row IDs: {patient_id}_{row_index}_{lead}
            row_ids = [f'{sample_id}_{x}_{lead}' for x in range(num_rows)]
            
            res.append(pd.DataFrame({
                'id': row_ids,
                'value': s
            }))
        
        if i % 10 == 0:
            gc.collect()
    
    # Combine all results
    submission = pd.concat(res, axis=0, ignore_index=True)
    
    # Validation
    print(f"\n📊 Submission Statistics:")
    print(f"  Total rows: {len(submission):,}")
    print(f"  Unique patients: {submission['id'].str.split('_').str[0].nunique()}")
    print(f"  Null values: {submission.isnull().sum().sum()}")
    
    # Check each patient
    for patient_id in test_ids:
        patient_rows = len(submission[submission['id'].str.startswith(f"{patient_id}_")])
        expected_rows = test[test['id'] == patient_id]['number_of_rows'].sum()
        print(f"  Patient {patient_id}: {patient_rows:,} rows (expected: {expected_rows:,})")
    
    # Save submission
    submission.to_csv('submission.csv', index=False)
    print(f"\n✅ Saved: submission.csv")
    
    # Show samples
    print(f"\nFirst 30 rows:")
    print(submission.head(30))
    
    print(f"\nLast 30 rows:")
    print(submission.tail(30))
    
    # Show sample for each patient
    for patient_id in test_ids:
        print(f"\n{patient_id} samples:")
        print(submission[submission['id'].str.startswith(f"{patient_id}_")].head(20))
    
    return submission

# ============================================================================
# MAIN TRAINING FUNCTION
# ============================================================================

def main():
    print("\n" + "="*80)
    print("TRAINING PIPELINE")
    print("="*80)
    
    config = {
        'img_size': (384, 384),
        'batch_size': 16,
        'num_epochs': 1,
        'learning_rate': 1e-4,
        'latent_dim': 256,
        'seq_length': 5000,
        'num_leads': 12,
        'num_workers': 4,
        'warmup_epochs': 2
    }
    
    print("\nConfiguration:")
    for k, v in config.items():
        print(f"  {k}: {v}")
    
    from sklearn.model_selection import train_test_split
    
    print(f"\nTotal patients available: {len(patient_dirs)}")
    train_ids, val_ids = train_test_split(patient_dirs, test_size=0.15, random_state=42)
    
    print(f"Train patients: {len(train_ids)}")
    print(f"Val patients: {len(val_ids)}")
    
    # Create datasets
    print("\nCreating datasets...")
    train_dataset = ECGDataset(base_path, train_ids, 
                               transform=get_transforms(True, config['img_size']),
                               max_len=config['seq_length'])
    
    val_dataset = ECGDataset(base_path, val_ids,
                            transform=get_transforms(False, config['img_size']),
                            max_len=config['seq_length'])
    
    train_loader = DataLoader(train_dataset, 
                             batch_size=config['batch_size'],
                             shuffle=True,
                             num_workers=config['num_workers'],
                             pin_memory=torch.cuda.is_available())
    
    val_loader = DataLoader(val_dataset,
                           batch_size=config['batch_size'],
                           shuffle=False,
                           num_workers=config['num_workers'],
                           pin_memory=torch.cuda.is_available())
    
    # Model
    print("\nInitializing model...")
    model = ECGModel(config['latent_dim'], config['seq_length'], config['num_leads']).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Parameters: {total_params:,}")
    
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)
    
    from torch.optim.lr_scheduler import CosineAnnealingLR
    scheduler = CosineAnnealingLR(optimizer, T_max=config['num_epochs'] - config['warmup_epochs'])
    
    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    
    # Training
    history = {'train_loss': [], 'val_loss': [], 'train_snr': [], 'val_snr': []}
    best_val_snr = -1000
    
    print("\n" + "="*80)
    print("STARTING TRAINING")
    print("="*80)
    
    for epoch in range(config['num_epochs']):
        print(f"\nEpoch {epoch+1}/{config['num_epochs']}")
        print("-" * 80)
        
        if epoch < config['warmup_epochs']:
            lr = config['learning_rate'] * (epoch + 1) / config['warmup_epochs']
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            print(f"Warmup LR: {lr:.6f}")
        
        train_loss, train_snr = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_loss, val_snr = validate(model, val_loader, criterion, device)
        
        if epoch >= config['warmup_epochs']:
            scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_snr'].append(train_snr)
        history['val_snr'].append(val_snr)
        
        print(f"\n📊 Epoch Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train SNR: {train_snr:.2f} dB")
        print(f"  Val Loss: {val_loss:.4f} | Val SNR: {val_snr:.2f} dB")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if not np.isnan(val_snr) and val_snr > best_val_snr:
            best_val_snr = val_snr
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"  ✅ Best model saved (SNR: {best_val_snr:.2f} dB)")
        
        if np.isnan(train_loss) or np.isnan(val_loss):
            print("\n⚠️  Training collapsed. Stopping.")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
    
    print(f"\n{'='*80}")
    print(f"✅ TRAINING COMPLETE")
    print(f"Best Val SNR: {best_val_snr:.2f} dB")
    print(f"{'='*80}")
    
    return model, history


# ============================================================================
# RUN PIPELINE
# ============================================================================

if __name__ == "__main__":
    # Check if test.csv exists
    test_csv_path = os.path.join(base_path, 'test.csv')
    if not os.path.exists(test_csv_path):
        print(f"❌ test.csv not found at {test_csv_path}")
    else:
        if len(patient_dirs) > 0:
            # Train model
            model, history = main()
            
            # Generate submission with CORRECT format
            submission = generate_submission_correct(model, base_path, device)
            
            print("\n" + "="*80)
            print("✅ PIPELINE COMPLETE")
            print("="*80)
        else:
            print("❌ No training data found!")
            print(f"  Patient dirs: {len(patient_dirs)}")

GPU: Tesla T4
ECG IMAGE TO TIME-SERIES - FULL FIXED VERSION

DATA VERIFICATION

train.csv loaded: (977, 3)
Columns: ['id', 'fs', 'sig_len']

test.csv loaded: (24, 4)
Columns: ['id', 'lead', 'fs', 'number_of_rows']
Sample:
           id lead    fs  number_of_rows
0  1053922973    I  1000            2500
1  1053922973   II  1000           10000
2  1053922973  III  1000            2500
3  1053922973  aVR  1000            2500
4  1053922973  aVL  1000            2500

Total patient folders: 977

🔍 TEST IMAGES FOUND: 2
  - 1053922973.png
  - 2352854581.png

TRAINING PIPELINE

Configuration:
  img_size: (384, 384)
  batch_size: 16
  num_epochs: 1
  learning_rate: 0.0001
  latent_dim: 256
  seq_length: 5000
  num_leads: 12
  num_workers: 4
  warmup_epochs: 2

Total patients available: 977
Train patients: 830
Val patients: 147

Creating datasets...
  📊 Total samples loaded: 7470
  📊 Total samples loaded: 1323

Initializing model...
Parameters: 2,529,676

STARTING TRAINING

Epoch 1/1
----------

Validation: 100%|██████████| 83/83 [02:58<00:00,  2.15s/it, loss=0.0080, snr=-0.00]



📊 Epoch Summary:
  Train Loss: 0.0095 | Train SNR: -0.01 dB
  Val Loss: 0.0098 | Val SNR: -0.00 dB
  LR: 0.000050
  ✅ Best model saved (SNR: -0.00 dB)

✅ TRAINING COMPLETE
Best Val SNR: -0.00 dB

GENERATING SUBMISSION - CORRECT FORMAT

Test CSV Info:
  Shape: (24, 4)
  Columns: ['id', 'lead', 'fs', 'number_of_rows']

Sample:
            id lead    fs  number_of_rows
0   1053922973    I  1000            2500
1   1053922973   II  1000           10000
2   1053922973  III  1000            2500
3   1053922973  aVR  1000            2500
4   1053922973  aVL  1000            2500
5   1053922973  aVF  1000            2500
6   1053922973   V1  1000            2500
7   1053922973   V2  1000            2500
8   1053922973   V3  1000            2500
9   1053922973   V4  1000            2500
10  1053922973   V5  1000            2500
11  1053922973   V6  1000            2500
12  2352854581    I  1000            2500
13  2352854581   II  1000           10000
14  2352854581  III  1000            2500


Inference:  50%|█████     | 1/2 [00:00<00:00,  3.54it/s]

  ✓ Processed 1053922973: shape (4, 5000)


Inference: 100%|██████████| 2/2 [00:00<00:00,  3.92it/s]


  ✓ Processed 2352854581: shape (4, 5000)

📝 Generating submission rows...


Creating submission: 100%|██████████| 2/2 [00:00<00:00, 10.43it/s]



📊 Submission Statistics:
  Total rows: 75,000
  Unique patients: 2
  Null values: 0
  Patient 1053922973: 37,500 rows (expected: 37,500)
  Patient 2352854581: 37,500 rows (expected: 37,500)

✅ Saved: submission.csv

First 30 rows:
                 id     value
0    1053922973_0_I -0.019550
1    1053922973_1_I -0.016185
2    1053922973_2_I -0.012820
3    1053922973_3_I -0.010497
4    1053922973_4_I -0.008175
5    1053922973_5_I -0.006594
6    1053922973_6_I -0.005014
7    1053922973_7_I -0.003968
8    1053922973_8_I -0.002922
9    1053922973_9_I -0.002249
10  1053922973_10_I -0.001576
11  1053922973_11_I -0.001153
12  1053922973_12_I -0.000730
13  1053922973_13_I -0.000469
14  1053922973_14_I -0.000209
15  1053922973_15_I -0.000050
16  1053922973_16_I  0.000108
17  1053922973_17_I  0.000203
18  1053922973_18_I  0.000298
19  1053922973_19_I  0.000355
20  1053922973_20_I  0.000411
21  1053922973_21_I  0.000445
22  1053922973_22_I  0.000479
23  1053922973_23_I  0.000499
24  1053922973_24_